In [1]:
from __future__ import print_function
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import pylab
import pandas as pd
import numpy as np
import os
import sys
import gzip
import itertools
import operator
import subprocess
import twobitreader
from Bio.Alphabet import IUPAC
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import pysam

from uditas.uditas_replace_helpers import *

#  Pipeline1
# This, for the most part, is the UDITAS pipeline modified to allow for REPLACE targeting. 

## There is a second pipeline below to do global alignments to quantify large deletions, translocatins, concatomers, etc by aligning the sequence past the break site to the entire genome+targeting sequence.

## Inputs
#### next few window are different inputs to make it run well

In [2]:
#things taken from original uditas args

check_plasmid_insertions = 1
ncpu = 4
window_size = 15
amplicon_window_around_cut = 1000
min_MAPQ = 5
min_AS = -180
process_AMP_seq_run = 0 #off

In [11]:
#Directory This is the same for all files from the same seq run

#this is a minimal directory of only ~300 files for quick debugging
directory = '/media/edanner/NewUbuntuSpace/Workspace/LinearAmp/Sequence2_191129_MN00157_0047_A000H2GWGF/P_Eric4_Tn5/10000Reads'

#directory = '/media/edanner/NewUbuntuSpace/Workspace/LinearAmp/Sequence2_191129_MN00157_0047_A000H2GWGF/P_Eric4_Tn5'

print(directory)

/media/edanner/NewUbuntuSpace/Workspace/LinearAmp/Sequence2_191129_MN00157_0047_A000H2GWGF/P_Eric4_Tn5/10000Reads


### make sure the following primer and downstream seq is unique and for the targeting exp

In [4]:
#primer sequences for mispriming. Primer sequence + 12nt downstream 
#this will only be avialable for read2

##### the sequences need to be capital for this to work########

#polb fwd 
print('this is for polb fwd (row 0 in sheet)')
primer_seq_plus_downstream ='TGGTTTTATTTCACCCCATGATAGTAATTATATCACT'


#print('this is for pipeline1 polb rev (row 1 in sheet)')
#primer_seq_plus_downstream ='ACAAAAGAGGCCAAGCTGGAGCAGGAAATAGATGC'


#print('this is for pipeline1 mCherryRev, row2 in sheet')
#make sure letters are Capital! (row 2 in sheet)
#primer_seq_plus_downstream ='GTTAGCAGACTTCCTCTGCCCTCAATCTTTAAGAA'


#print('this is for mCherryFWD, row3 in sheet')
#primer_seq_plus_downstream ='GCAGGAGCTCGTCGACCCATGGGGGCCCGCCCC'


#check what is printed below!!!

this is for polb fwd (row 0 in sheet)


### Sheets information input (change for each targeting exp)

In [5]:
# Function to make the 'amplicon_info' list. Taking the line of our experiment csv file

def get_csv_data(dir_sample, line_of_data_from_csv):
    sample_info_filename = os.path.join(dir_sample, 'sample_info.csv')
    experiments = pd.read_csv(sample_info_filename)
    return experiments.loc[line_of_data_from_csv]


#the amplicon info is related to the line on the csv file. It is indexed from 0. For PolbF we use 0.
#it is on my extera space so need to make sure that is mounted. (first thing to check if it throws and error)

#0 is because this polbFwd is in the first line. Need to change.
amplicon_info = get_csv_data(directory, 0)

#1 is because this polbRev 
#amplicon_info = get_csv_data(directory, 1)

#2 is because this mcherry_rev
#amplicon_info = get_csv_data(directory, 2)

#3 is because this mcherry_fwd
#amplicon_info = get_csv_data(directory, 3)


###########check waht is printed below #######
amplicon_info

NGS_req-ID                                                                     A000H2GWGF
name                                                                        Tn5_Pol50_pbF
Sample                                                                               Polb
description                                   MiniSeq_K562_PolbHIROS_50per_unsorted_polbF
Control sample (Y/N)                                                                    N
Notes                                                                                 NaN
Dilution                                                                              NaN
Cell name_type                                                                       K562
UMI_Len                                                                        NNNNNNNNNN
IndexI7Primer                                                                      prE368
I7_Index_ID                                                          P7_N701_SBS12nextera
index_I1  

### Set global Variables

In [6]:
# Assign the file_genome_2bit location. This is needed for pulling sequence from the referene genome by location
assembly = amplicon_info['genome']
file_genome_2bit = os.path.join('/media/edanner/NewUbuntuSpace/Workspace/Ref_Genomes', assembly + '.2bit')
print(file_genome_2bit)

# BOWTIE2_INDEXES are needed for global alignments
#not sure if this will work
#normally in bash: export BOWTIE2_INDEXES=/media/edanner/NewUbuntuSpace/Workspace/Ref_Genomes
#check in bash: > ECHO $GENOMES_2BIT

#this is the refrence genome for Bowtie from Illumina. It does not have our targeting vector
%env BOWTIE2_INDEXES=/media/edanner/NewUbuntuSpace/Workspace/Ref_Genomes

#check the env variables
#%env

/media/edanner/NewUbuntuSpace/Workspace/Ref_Genomes/hg38.2bit
env: BOWTIE2_INDEXES=/media/edanner/NewUbuntuSpace/Workspace/Ref_Genomes


### Functions to process the reads

In [22]:
#I imported them from my helper file to keep this space cleaner

### Discard Mispriming Reads
When you put a universal primer on the ends of everything, every mispriming event will amplify. An effect we normally don't deal with. I did nested PCR to reduce this. However 85% of the alignments in the UDITAS data I looked at seemed to be mispriming. They did all their blasting and analysis before removing mispriming. But to save computational power and remove error early on I will discard mispriming events. 
They discard these only for plasmid alignments analyze_alignments_plasmid for some reason which comes from the bam file.

By eye it looks like 50-90% of my reads are correctly primed which is amaizng. Nesting helepd a lot.


In [23]:
correct_priming(directory, amplicon_info, primer_seq_plus_downstream)



reads_in_experiment_list_count 323
reads_not_in_experiment_list_count 30


### Trim off illumina adapter from shorter reads

In [24]:
#Running the trimming

trim_fastq(directory, amplicon_info, 0)



In [25]:
# Running the reference plasmid creation
create_plasmid_reference(directory, amplicon_info)


In [26]:
#Test the get_reaction_type is working
#uditas_replace_helpers.get_reaction_type(amplicon_info)
get_reaction_type(amplicon_info)

'replace'

In [27]:
#Make the amplicons by calling the create_amplicon function

create_amplicon(directory, amplicon_info, file_genome_2bit)

In [28]:
#try out the alignment to the plasmid
align_plasmid_local(directory, amplicon_info, ncpu=4)


In [29]:
#extract the unmapped reads
extract_unmapped_reads_plasmid(directory, amplicon_info)

In [30]:
#run the plasmid analysis to coudn plasmid integration events
result_plasmid_df = analyze_alignments_plasmid(directory, amplicon_info, min_MAPQ, file_genome_2bit, True)
result_plasmid_df

,target_plus_plasmid_total_reads,target_plus_plasmid_total_reads_collapsed,plasmid_only_total_reads,plasmid_only_total_reads_collapsed
0,0,0,0,0


In [31]:
#align against our suite of amplicons
# took about 5 min with 300k reads
align_amplicon(directory, amplicon_info, check_plasmid_insertions, ncpu)

In [32]:
#this will extract unmap reads new folder (files that did not align to the predicted structural variants)
extract_unmapped_reads_amplicons(directory, amplicon_info)

## This section is to do the analysis of the amplicon alignments

In [33]:
result_reads_in_all_amplicons_df = analyze_alignments_all_amplicons(directory, amplicon_info, min_MAPQ, min_AS)
result_reads_in_all_amplicons_df

number of reads before mispriming cleanup 353
number of unique umis before mispriming cleanup 160


,all_amplicons_total_reads,all_amplicons_total_reads_collapsed
0,226,86


In [34]:
#took 
result_amplicon_df = analyze_alignments(directory, amplicon_info, window_size, amplicon_window_around_cut, min_MAPQ, min_AS)
result_amplicon_df

,wt_cut1_total_reads,wt_cut1_total_indels,wt_cut1_total_deletions,wt_cut1_total_insertions,wt_cut1_total_reads_collapsed,wt_cut1_total_indels_collapsed,wt_cut1_total_deletions_collapsed,wt_cut1_total_insertions_collapsed,wt_cut2_total_reads,wt_cut2_total_indels,...,1a_1a_cut1_total_insertions_collapsed,2b_2b_cut1_total_reads,2b_2b_cut1_total_indels,2b_2b_cut1_total_deletions,2b_2b_cut1_total_insertions,2b_2b_cut1_total_reads_collapsed,2b_2b_cut1_total_indels_collapsed,2b_2b_cut1_total_deletions_collapsed,2b_2b_cut1_total_insertions_collapsed,median_fragment_size
0,37,21,6,15,12,6,2,4,0,0,...,0,0,0,0,0,0,0,0,0,208.0


### This is the Global alignment of the remaining reads

#### What I need here is to modify it so that 

In [35]:
#Run the allignment
assembly = amplicon_info['genome']
align_genome_global(directory, amplicon_info, assembly)

In [36]:
#Do the global alignment analysis
results_genome_global_df = analyze_alignments_genome_global(directory, amplicon_info, min_MAPQ, min_AS,  file_genome_2bit)
results_genome_global_df



the number of alignments that were attempted to align 0


,genomewide_total_reads,genomewide_total_reads_collapsed,genomewide_target_only_reads,genomewide_target_only_reads_collapsed
0,0,0,0,0


### Summerize data

In [37]:
read_count = pd.DataFrame()
read_count.loc[0,'read_count'] = count_reads(directory, amplicon_info)

In [38]:
#summerize the data

summary_all_alignments = get_summary_all_alignments(directory, amplicon_info, read_count.loc[[0]], result_plasmid_df, result_reads_in_all_amplicons_df, results_genome_global_df)
summary_all_alignments

,read_count,target_plus_plasmid_total_reads,target_plus_plasmid_total_reads_collapsed,plasmid_only_total_reads,plasmid_only_total_reads_collapsed,all_amplicons_total_reads,all_amplicons_total_reads_collapsed,genomewide_total_reads,genomewide_total_reads_collapsed,genomewide_target_only_reads,genomewide_target_only_reads_collapsed,total_aligned,total_aligned_collapsed,percent_aligned,percent_aligned_all_amplicons
0,323.0,0,0,0,0,226,86,0,0,0,0,226,86,69.96904,100.0


In [39]:
results_alignments_junction = pd.concat([result_amplicon_df, result_plasmid_df], axis=1)

sample_info_filename = os.path.join(directory, 'sample_info.csv')

experiments = pd.read_csv(sample_info_filename)


results_summary = summarize_results(results_alignments_junction)
results_summary
#this outputs it in a directory above the N501_701 folder as a summary file. Need to label it or it will be overwritten by next tone
results_summary_with_experiments = pd.concat([experiments, results_summary], axis=1)
results_summary_with_experiments.to_excel(os.path.join(directory, 'results_summary.xlsx'))

results_pivot = melt_results(results_summary_with_experiments)
results_pivot.to_excel(os.path.join(directory, 'results_summary_pivot.xlsx'))
results_pivot

,NGS_req-ID,name,Sample,description,Control sample (Y/N),Notes,Dilution,Cell name_type,UMI_Len,IndexI7Primer,...,doner_tail_tail_cut1_total_reads_percent,doner_head_tail_cut1_total_reads_percent,doner_head_head_cut1_total_reads_percent,1a_1a_cut1_total_reads_percent,2b_2b_cut1_total_reads_percent,target_plus_plasmid_total_reads_percent,plasmid_only_total_reads_percent,total_aligned_junctions_collapsed,Type,Percent Editing
0,A000H2GWGF,Tn5_Pol50_pbF,Polb,MiniSeq_K562_PolbHIROS_50per_unsorted_polbF,N,NaN,NaN,K562,NNNNNNNNNN,prE368,...,0.0,0.0,0.0,1.111111,0.0,0.0,0.0,68.0,wt_cut1_total_reads_collapsed_percent,17.647059
1,A000H2GWGF,Tn5_Pol50_pbR,Polb,MiniSeq_K562_PolbHIROS_50per_unsorted_polbR,N,NaN,NaN,K562,NNNNNNNNNN,prE369,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,wt_cut1_total_reads_collapsed_percent,NaN
2,A000H2GWGF,Tn5_Pol50_mCh,Polb,MiniSeq_K562_PolbHIROS_50per_unsorted_mCherryp...,N,NaN,NaN,K562,NNNNNNNNNN,prE370,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,wt_cut1_total_reads_collapsed_percent,NaN
3,A000H2GWGF,Tn5_Pol50_bpa,Polb,MiniSeq_K562_PolbHIROS_50per_unsorted_mCherry_Fwd,N,NaN,NaN,K562,NNNNNNNNNN,prE371,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,wt_cut1_total_reads_collapsed_percent,NaN
4,A000H2GWGF,Tn5_WT_pbF,Polb,MiniSeq_K562_PolbHIROS_Wtcontaminated_w_50per_...,N,NaN,NaN,K562,NNNNNNNNNN,prE372,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,wt_cut1_total_reads_collapsed_percent,NaN
5,A000H2GWGF,Tn5_WT_pbR,Polb,MiniSeq_K562_PolbHIROS_Wtcontaminated_w_50per_...,N,NaN,NaN,K562,NNNNNNNNNN,prE373,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,wt_cut1_total_reads_collapsed_percent,NaN
6,A000H2GWGF,Tn5_Pol50_pbF,Polb,MiniSeq_K562_PolbHIROS_50per_unsorted_polbF,N,NaN,NaN,K562,NNNNNNNNNN,prE368,...,0.0,0.0,0.0,1.111111,0.0,0.0,0.0,68.0,wt_cut2_total_reads_collapsed_percent,0.000000
7,A000H2GWGF,Tn5_Pol50_pbR,Polb,MiniSeq_K562_PolbHIROS_50per_unsorted_polbR,N,NaN,NaN,K562,NNNNNNNNNN,prE369,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,wt_cut2_total_reads_collapsed_percent,NaN
8,A000H2GWGF,Tn5_Pol50_mCh,Polb,MiniSeq_K562_PolbHIROS_50per_unsorted_mCherryp...,N,NaN,NaN,K562,NNNNNNNNNN,prE370,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,wt_cut2_total_reads_collapsed_percent,NaN
9,A000H2GWGF,Tn5_Pol50_bpa,Polb,MiniSeq_K562_PolbHIROS_50per_unsorted_mCherry_Fwd,N,NaN,NaN,K562,NNNNNNNNNN,prE371,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,wt_cut2_total_reads_collapsed_percent,NaN


# Pipeline2
#  This is a second pipeline. Different than Uditas but built in a simliar form. 
## It uses the read distal to the gene specic primer. This read covers the region downstream of the break. After clipping off anything remaing upstream of the break we make a global alignment to look at genomic integrations and translocations. This also quantifies the alignments.


### 0. Use correct_prime function in pipeline1 to arrive at reads that were correctly primed.
### 1. Trim correctly primed reads to the break site  
### 2. Only use a single end of the pair end reads (distal gene specific primer)
### 3. Use bowtie2 genome that has the targeting seq in it added
### 4. Do global end-to-end alignment
### 5. Do a local alignment
### 6. process bam file and pull out the high mapQ reads

The idea here is to trim the sequence up to the cut on both reads and then do a global alignment end-to-end agasint the human genome.

The idea here is to take the files that are 'trimmed' and 'correct priming' checking. Then to do another round of trimming to get down to the cut site. I need to make another folder for 'trimmed_to_cut'

For Tn5 setup the 'Read 1' contains the sequence away from the gene specific primer. So it is the read we should concentration on if we don't do paired end.

For LAM the 'Read 2' is the the sequence distal to the gene specific primer

### Inputs

In [7]:
#This is for the sequence to the breaksite for the pipeline of genomic alignment 
# make sure the sequences are capital


print('this is for pipeline2 polb fwd (row 0 in sheet)')
seq_primer_to_breaksite = 'TGGTTTTATTTCACCCCATGATAGTAATTATATCACTTCTGATCTGTTAAGAATAGACCTTTTAAAAGTATTGGATAACTTAGAGATGAGACATCTTCAGTTACTCTGTTATTCACCTATTACTCCTTAGGTTACTTGTGAATAATTTTGTGTGGGTCA'

#print('this is for pipeline2 polb rev (row 1 in sheet)')
#seq_primer_to_breaksite = 'ACAAAAGAGGCCAAGCTGGAGCAGGAAATAGATGCACACGGAGGAAATGGTAGTGGAGTTCAGAGGAGGGAGAGGTTCTTTCTGCCTGGG'

#this has some seq homology to the genome but should be ok for this pipeline trimming
#print('this is for pipeline2 mCherryRev, row2 in sheet')
#seq_primer_to_breaksite = 'GTTAGCAGACTTCCTCTGCCCTCAATCTTTAAGAAAAAAAAAAGTCTAACAATGATTTAGGAATGCTTTGAGGACTTAAATGATCTTATTGGAAACATACCAGTCTGCTAAAAGACTAATTTTGTGTGGGTCA'

#this is more reliable than the mCherry rev as it has no sequence homology to the genome
#print('this is for pipeline2 mCherryFWD, row3 in sheet')
#seq_primer_to_breaksite = 'GCAGGAGCTCGTCGACCCATGGGGGCCCGCCCCAACTGGGGTAACCTTTGGGCTCCCCGGGCGCGACTAGTGAATTCAGATCTGATATCTCTAGAAGTCCTGGG'

#check amplicon info and directory
print(directory)
amplicon_info

this is for pipeline2 polb fwd (row 0 in sheet)
/media/edanner/NewUbuntuSpace/Workspace/LinearAmp/Sequence2_191129_MN00157_0047_A000H2GWGF/P_Eric4_BCL2Fastq_only/10000Reads


NGS_req-ID                                                                     A000H2GWGF
name                                                                        Tn5_Pol50_pbF
Sample                                                                               Polb
description                                   MiniSeq_K562_PolbHIROS_50per_unsorted_polbF
Control sample (Y/N)                                                                    N
Notes                                                                                 NaN
Dilution                                                                              NaN
Cell name_type                                                                       K562
UMI_Len                                                                        NNNNNNNNNN
IndexI7Primer                                                                      prE368
I7_Index_ID                                                          P7_N701_SBS12nextera
index_I1  


## Need to make an new bowtie2 index file that includes targeting for alignment agains the whole genome so it is all in one sheet together. 

### Other option would be to align it to the amplicon, extract unaligned files and then align to the genome but seems cleaner this way. 
#### Ideally every sequence is unique between the targeting vector and genome.


1. Build your fastas of interest and label .fa files.
    1. You need fasta of hg38 or reference genome. You can pull this from downloaded bowtie indexed sampels and then use the following command to turn the index into a fasta file: bowtie2-inspect hg38 > hg38.fa   
    2. Put all the fasta files in the same folder
2. index the files with bowtie
    1. use the command bowtie2-build -f pe038_mc.fa,hg38.fa -p hg38_plus_targetvector
    2. this has the -p to make it take less ram in my case.
    3. In this case it adds the hg38 and the minicircle targeting file together
    4. I have a Intel® Core™ i7-5500U CPU @ 2.40GHz × 4 with 15.1 GiB ram and it required about 13.8 gigs of ram and 2 hours to do a hg38+small fasta index
    5. be sure to pay attention to the name of the new indexed file. "hg38_plus_targetvector" in example above. Add it to the sample_info.csv sheet. Under the tab 'genome_plus_targeting'.
    6. you can check it indexed correcctly: bowtie2-inspect -s hg38_plus_targetvector
    

    



In [8]:
# Assign the file_genome_2bit location. This is needed for pulling sequence from the referene genome by location
#need a new build
assembly_plus_targetvector = amplicon_info['genome_plus_targeting']
file_genome_2bit_plus_target = os.path.join('/media/edanner/NewUbuntuSpace/Workspace/Ref_Genomes', assembly_plus_targetvector + '.2bit')
print(file_genome_2bit_plus_target)

#Needs to have a bowtie build in which the targeting vector is included!!

%env BOWTIE2_INDEXES=/media/edanner/NewUbuntuSpace/Workspace/Ref_Genomes

#check the env variables
#%env

/media/edanner/NewUbuntuSpace/Workspace/Ref_Genomes/hg38_plus_targetvector.2bit
env: BOWTIE2_INDEXES=/media/edanner/NewUbuntuSpace/Workspace/Ref_Genomes


In [9]:
#Trim the break up to the cut site

trim_fastq_to_break(directory, amplicon_info, seq_primer_to_breaksite)

In [19]:
#this aligns everything after the break to end-to-end 
align_afterbreak_end_to_end_genome_global(directory, amplicon_info, assembly_plus_targetvector, keep_sam=0)

In [10]:
#this aligns everything in the local format wehre it can soft clip the ends
align_afterbreaks_genome_local(directory, 1, amplicon_info, assembly_plus_targetvector, keep_sam=0)